In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem, DataStructs
import numpy as np
from rdkit.Chem import (
    PandasTools,
    Draw,
    Descriptors,
    MACCSkeys,
    rdFingerprintGenerator,
)
import math
import keras_tuner
import tensorflow as tf
from tensorflow import keras
#import dropout
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

2024-08-29 11:50:38.060347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 11:50:38.134696: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 11:50:38.155173: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-29 11:50:38.297319: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 11:50:39.580131: W tensorflow/compiler/tf2

In [2]:
beta = pd.read_csv('beta_activity_class.csv') #Clean CSV file with beta secretase smiles and activity

In [3]:
beta["activity_class"].value_counts()
#dropna of activity_class
beta = beta.dropna(subset=["activity_class"])

In [4]:
#Create morgan fingerprints for compound smiles using RDKit (refer back to exploratory_DA)
rd_mols = [Chem.MolFromSmiles(smiles) for smiles in beta["Ligand SMILES"]]
beta["RDkit Molecule"] = rd_mols
del rd_mols
beta.dropna(subset=["RDkit Molecule"], inplace=True)
morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)
morgan = [morgan_gen.GetFingerprint(mol) for mol in beta["RDkit Molecule"]]
tmpX = [list(m) for m in morgan]
print(len(tmpX))

[11:50:42] Explicit valence for atom # 18 N, 4, is greater than permitted


13154


In [5]:
np.unique(beta["activity_class"].values)

array(['inactive', 'moderately_active', 'very_active'], dtype=object)

In [6]:
#transfor activity_class to 0,1,2
beta["activity_class"] = beta["activity_class"].replace("moderately_active", 1)
beta["activity_class"] = beta["activity_class"].replace("inactive", 0)
beta["activity_class"] = beta["activity_class"].replace("very_active", 2)

/tmp/ipykernel_40507/193602242.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  beta["activity_class"] = beta["activity_class"].replace("very_active", 2)


In [7]:
beta["activity_class"].value_counts()

activity_class
1    9286
2    2182
0    1686
Name: count, dtype: int64

In [8]:
tmp = beta["activity_class"].values

In [9]:
from sklearn.model_selection import train_test_split
Y = tmp
print(len(Y))
X = pd.DataFrame(tmpX)
print(len(X))
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.2, train_size=0.8,random_state=42)
#Y_train


13154
13154


In [10]:
np.unique(Y, return_counts=True)

(array([0, 1, 2]), array([1686, 9286, 2182]))

In [68]:
#Train three non-neural net models for classification 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report



rf_model = RandomForestClassifier(n_estimators=100, random_state=42)


rf_model.fit(X_train, Y_train)
cv_scores = cross_val_score(rf_model, X_train, Y_train, cv=5)
print(f'Cross-validation scores: {cv_scores}')
print(f'Mean CV score: {cv_scores.mean()}')
# Predict and evaluate
y_pred = rf_model.predict(X_test)
print(classification_report(Y_test, y_pred))


Cross-validation scores: [0.83182898 0.81045131 0.81852732 0.81178707 0.79847909]
Mean CV score: 0.8142147521291871
              precision    recall  f1-score   support

           0       0.76      0.65      0.70       358
           1       0.85      0.92      0.88      1838
           2       0.75      0.59      0.66       435

    accuracy                           0.83      2631
   macro avg       0.79      0.72      0.75      2631
weighted avg       0.82      0.83      0.82      2631



In [70]:
from sklearn.svm import SVC
svm_model = SVC(kernel='rbf', C=1.0, gamma='auto')
svm_model.fit(X_train, Y_train)
y_pred = svm_model.predict(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       358
           1       0.70      1.00      0.82      1838
           2       1.00      0.01      0.03       435

    accuracy                           0.70      2631
   macro avg       0.57      0.34      0.28      2631
weighted avg       0.65      0.70      0.58      2631



/home/scro4068/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/scro4068/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/scro4068/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [71]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(penalty='l2', C=1.0, random_state=42, max_iter=200)
lr_model.fit(X_train, Y_train)

y_pred = lr_model.predict(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.64      0.69       358
           1       0.85      0.91      0.88      1838
           2       0.72      0.61      0.66       435

    accuracy                           0.82      2631
   macro avg       0.77      0.72      0.74      2631
weighted avg       0.81      0.82      0.82      2631



/home/scro4068/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [72]:
#measure accuracy of the models
from sklearn.metrics import accuracy_score
rf_acc = accuracy_score(Y_test, rf_model.predict(X_test))
svm_acc = accuracy_score(Y_test, svm_model.predict(X_test))
lr_acc = accuracy_score(Y_test, lr_model.predict(X_test))
#print
print(f'Random Forest Accuracy: {rf_acc}')
print(f'SVM Accuracy: {svm_acc}')
print(f'Logistic Regression Accuracy: {lr_acc}')


Random Forest Accuracy: 0.8259217027746104
SVM Accuracy: 0.700874192322311
Logistic Regression Accuracy: 0.8202204484986697


In [11]:
from keras.models import Sequential
from keras.layers import Dense
#Neural net model for classification (you can refer to https://psrivasin.medium.com/multiclass-classification-using-tensorflow-850ee138d0ca for a guide)
#Input layer from morgan fingerprints
NN_model = Sequential()
NN_model.add(Dense(256,kernel_initializer='normal', activation='relu', input_dim = X_train.shape[1]))
#hidden layer > hyperparameter tune 

#hyperparameter to tune: number of hidden layers (1 to 4), number of units in each layer (32-256), learning rate ( 1e-05 to 5e-05)
#dropout layer > tune dropout rate (0.1 to 0.3)
#Output 1 dense layer with 3 units, softmax activation
NN_model.add(Dense(3, activation='softmax'))

/home/scro4068/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1724928656.482828   40507 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724928656.639998   40507 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724928656.640175   40507 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724928656.646949   40507 cuda_executor.cc:1001] cou

In [12]:
def build(hp):
    num_layer = hp.Int('num_layer', min_value=1, max_value=4, step=1)
    num_units = hp.Int('num_units', min_value=32, max_value=512, step=16)
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.3, step=0.05)
    learning_rate = hp.Float('learning_rate', min_value=1e-05, max_value =5e-05, step = 1e-05)
    model = Sequential()
    model.add(Dense(num_units,kernel_initializer='normal', activation='relu', input_dim = X_train.shape[1]))
    for i in range(num_layer):
        model.add(Dense(num_units, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [18]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build,
    objective="val_loss",
    max_trials=30,
    executions_per_trial=1,
    overwrite=True,
    directory="hyper_tuning",
    project_name="experiment",
)

In [19]:
tuner.search_space_summary()


Search space summary
Default search space size: 4
num_layer (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': 'linear'}
num_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 16, 'sampling': 'linear'}
dropout_rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.3, 'step': 0.05, 'sampling': 'linear'}
learning_rate (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 5e-05, 'step': 1e-05, 'sampling': 'linear'}


In [20]:
trainX = np.array(X_train)
testX = np.array(X_test)
trainY = np.array(Y_train)
testY = np.array(Y_test)

In [21]:
import gc
gc.collect()

2855

In [22]:
tuner.search(x=trainX, y=trainY, validation_data=[testX,testY], epochs=20, callbacks=[keras.callbacks.TensorBoard("tensorboard/tb_logs")])

Trial 30 Complete [00h 00m 35s]
val_loss: 0.42199137806892395

Best val_loss So Far: 0.41263505816459656
Total elapsed time: 00h 19m 22s


In [23]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

/home/scro4068/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 304)            │       622,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 304)            │        92,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 304)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           915 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 716,531 (2.73 MB)

 Trainable params: 716,531 (2.73 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
best_model.save('best_model.keras')

In [25]:
best_model.fit(trainX, trainY, epochs=100, validation_data=(testX, testY))

Epoch 1/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8989 - loss: 0.2508 - val_accuracy: 0.8350 - val_loss: 0.4165
Epoch 2/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9056 - loss: 0.2365 - val_accuracy: 0.8328 - val_loss: 0.4229
Epoch 3/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9028 - loss: 0.2366 - val_accuracy: 0.8362 - val_loss: 0.4193
Epoch 4/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9050 - loss: 0.2329 - val_accuracy: 0.8377 - val_loss: 0.4252
Epoch 5/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9082 - loss: 0.2272 - val_accuracy: 0.8373 - val_loss: 0.4240
Epoch 6/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9112 - loss: 0.2184 - val_accuracy: 0.8354 - val_loss: 0.4228
Epoch 7/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9189 - loss: 0.2042 - val_accuracy: 0.8369 - val_loss: 0.4283
Epoch 8/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9161 - loss: 0.2033 - val_accu

KeyboardInterrupt: 